# Aircraft Landing Patterns Analysis

This notebook performs exploratory data analysis, trend analysis, forecasting, and anomaly detection on aircraft landing data from San Francisco International Airport.

## Table of Contents
1. Data Loading
2. Exploratory Data Analysis
3. Trend Analysis
4. Forecasting with Prophet
5. Anomaly Detection
6. Model Saving

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error
from prophet import Prophet
import joblib
import pickle
import os

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

## 1. Data Loading

Load the cleaned aircraft landing data.

In [ ]:
# Load cleaned data
df = pd.read_csv('../data/cleaned_data.csv')
df['date'] = pd.to_datetime(df['date'])

# Display first few rows
df.head()

## 2. Exploratory Data Analysis

Explore the data with summary statistics and visualizations.

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(df.describe())

# Aggregate monthly landings
monthly_landings = df.groupby('date')['landings'].sum().reset_index()

# Plot time series
plt.figure(figsize=(12, 6))
plt.plot(monthly_landings['date'], monthly_landings['landings'])
plt.title('Monthly Aircraft Landings Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Landings')
plt.show()

# Distribution of landings
plt.figure(figsize=(8, 5))
sns.histplot(df['landings'], bins=50, kde=True)
plt.title('Distribution of Landing Counts')
plt.show()

# Aircraft body types
if 'Aircraft Body Type' in df.columns:
    plt.figure(figsize=(10, 6))
    df['Aircraft Body Type'].value_counts().plot(kind='bar')
    plt.title('Aircraft Body Types')
    plt.xticks(rotation=45)
    plt.show()

## 3. Trend Analysis

Perform linear regression to analyze trends in landing data.

In [ ]:
# Prepare data for regression
monthly_landings['days'] = (monthly_landings['date'] - monthly_landings['date'].min()).dt.days
X = monthly_landings[['days']]
y = monthly_landings['landings']

# Fit linear regression
trend_model = LinearRegression()
trend_model.fit(X, y)
predictions = trend_model.predict(X)

# Plot trend
plt.figure(figsize=(12, 6))
plt.scatter(monthly_landings['date'], y, label='Actual')
plt.plot(monthly_landings['date'], predictions, color='red', label='Trend Line')
plt.title('Landing Trend Analysis')
plt.xlabel('Date')
plt.ylabel('Landings')
plt.legend()
plt.show()

# Print metrics
slope = trend_model.coef_[0]
mse = mean_squared_error(y, predictions)
print(f"Trend slope: {slope:.2f} landings per day")
print(f"Mean Squared Error: {mse:.2f}")

# Save model
os.makedirs('../models', exist_ok=True)
joblib.dump(trend_model, '../models/trend_model.pkl')
print("Trend model saved.")

## 4. Forecasting with Prophet

Use Facebook Prophet for time series forecasting.

In [ ]:
# Prepare data for Prophet
prophet_df = monthly_landings.rename(columns={'date': 'ds', 'landings': 'y'})

# Fit Prophet model
prophet_model = Prophet()
prophet_model.fit(prophet_df)

# Forecast next 12 months
future = prophet_model.make_future_dataframe(periods=12, freq='ME')
forecast = prophet_model.predict(future)

# Plot forecast
fig = prophet_model.plot(forecast)
plt.title('Landing Volume Forecast')
plt.show()

# Save model
with open('../models/prophet_model.pkl', 'wb') as f:
    pickle.dump(prophet_model, f)
print("Prophet model saved.")

## 5. Anomaly Detection

Detect anomalies in landing data using Isolation Forest.

In [ ]:
# Anomaly detection
anomaly_model = IsolationForest(contamination=0.1, random_state=42)
monthly_landings['anomaly'] = anomaly_model.fit_predict(monthly_landings[['landings']])

# Plot anomalies
plt.figure(figsize=(12, 6))
plt.plot(monthly_landings['date'], monthly_landings['landings'], label='Landings')
anomalies = monthly_landings[monthly_landings['anomaly'] == -1]
plt.scatter(anomalies['date'], anomalies['landings'], color='red', label='Anomalies', s=50)
plt.title('Anomaly Detection in Monthly Landings')
plt.xlabel('Date')
plt.ylabel('Landings')
plt.legend()
plt.show()

print(f"Number of anomalies detected: {len(anomalies)}")

# Save model
joblib.dump(anomaly_model, '../models/anomaly_model.pkl')
print("Anomaly model saved.")

## 6. Conclusion

This notebook demonstrated:
- Data loading and exploration
- Trend analysis with linear regression
- Time series forecasting with Prophet
- Anomaly detection with Isolation Forest

All models have been saved to the `models/` folder for future use.